source: https://opendata.hhs.gov/datasets/medicaid-provider-spending/

In [62]:
import pandas as pd
pd.set_option('display.max_rows', None) # no row limit

import pyarrow
import fastparquet



# below loads the data but takes ~2.5 minutes

In [100]:
import pyarrow.dataset as ds

#### UNCOMMENT IN CASE YOU NEED TO RUN AGAIN ####

# dataset = ds.dataset(
#     "/Users/luketaylor/Desktop/Pycharm/Datasets/medicaid-provider-spending.parquet",
#     format="parquet"
# )
# table = dataset.to_table()
# medicaid_provider_spending = table.to_pandas()

# medicaid_provider_spending['CLAIMS_PER_BENEFICIARY'] = medicaid_provider_spending['TOTAL_CLAIMS'] / medicaid_provider_spending['TOTAL_UNIQUE_BENEFICIARIES']
# medicaid_provider_spending['PAID_PER_CLAIM'] = medicaid_provider_spending['TOTAL_PAID'] / medicaid_provider_spending['TOTAL_CLAIMS']
# medicaid_provider_spending.head()

NameError: name 'medicaid_provider_spending' is not defined

In [99]:
print('len df:')
print(len(medicaid_provider_spending))
medicaid_provider_spending.columns

len df:


NameError: name 'medicaid_provider_spending' is not defined

# load in hcpc options

In [38]:
hcpc_mapping_2026 = pd.read_csv('/Users/luketaylor/Desktop/Pycharm/Datasets/2026_HCPC_Code_List.csv')
hcpc_mapping_2026.head()

,HCPCS_CODE,HCPCS_DESCRIPTION,HCPCS_CATEGORY
0,86890,Autologous blood process,CLINICAL LABORATORY SERVICES
1,86891,Autologous blood op salvage,CLINICAL LABORATORY SERVICES
2,86927,Plasma fresh frozen,CLINICAL LABORATORY SERVICES
3,86930,Frozen blood prep,CLINICAL LABORATORY SERVICES
4,86931,Frozen blood thaw,CLINICAL LABORATORY SERVICES


groupby options:
- single groupby
- df.groupby(['col_1', 'col_2']).agg({
    'col_3': 'sum',
    'col_4': 'mean'
})


- groupby multiple columns
df.groupby(['col_1', 'col_2']).agg({
    'col_3': ['sum', 'count'],
    'col_4': ['mean', 'max']
})
# takes ~30 seconds to run

In [202]:
hcpc_by_month = df.groupby(['HCPCS_CODE', 'CLAIM_FROM_MONTH']).agg({
    'TOTAL_UNIQUE_BENEFICIARIES': 'sum',
    'TOTAL_CLAIMS': 'sum',
    'TOTAL_PAID': 'sum'
})

hcpc_by_month.reset_index(inplace=True)

hcpc_by_month['CLAIMS_PER_BENEFICIARY'] = hcpc_by_month['TOTAL_CLAIMS'] / hcpc_by_month['TOTAL_UNIQUE_BENEFICIARIES']
hcpc_by_month['PAID_PER_CLAIM'] = hcpc_by_month['TOTAL_PAID'] / hcpc_by_month['TOTAL_CLAIMS']
hcpc_by_month['PAID_PER_BENEFICIARY'] = hcpc_by_month['TOTAL_PAID'] / hcpc_by_month['TOTAL_UNIQUE_BENEFICIARIES']
hcpc_by_month.head()

KeyboardInterrupt: 

# todo:
- incorporate paid per beneficiary per month

In [40]:
hcpc_mapping_2026.head()

,HCPCS_CODE,HCPCS_DESCRIPTION,HCPCS_CATEGORY
0,86890,Autologous blood process,CLINICAL LABORATORY SERVICES
1,86891,Autologous blood op salvage,CLINICAL LABORATORY SERVICES
2,86927,Plasma fresh frozen,CLINICAL LABORATORY SERVICES
3,86930,Frozen blood prep,CLINICAL LABORATORY SERVICES
4,86931,Frozen blood thaw,CLINICAL LABORATORY SERVICES


# try and join to bring in descriptions

In [43]:
hcpc_w_descript_by_month = hcpc_by_month.merge(hcpc_mapping_2026, how='left', on=['HCPCS_CODE'])
hcpc_w_descript_by_month=hcpc_w_descript_by_month[['HCPCS_CATEGORY','HCPCS_DESCRIPTION','HCPCS_CODE', 'CLAIM_FROM_MONTH','TOTAL_UNIQUE_BENEFICIARIES','TOTAL_CLAIMS','TOTAL_PAID','CLAIMS_PER_BENEFICIARY','PAID_PER_CLAIM']]
hcpc_w_descript_by_month.head(10000)

,HCPCS_CATEGORY,HCPCS_DESCRIPTION,HCPCS_CODE,CLAIM_FROM_MONTH,TOTAL_UNIQUE_BENEFICIARIES,TOTAL_CLAIMS,TOTAL_PAID,CLAIMS_PER_BENEFICIARY,PAID_PER_CLAIM
0,NaN,NaN,1286A,2022-02,355,309532,43321.27,871.921127,0.139957
1,NaN,NaN,1286Z,2022-02,1906,1607071,229965.29,843.164218,0.143096
2,NaN,NaN,1286C,2022-02,202,157971,24226.68,782.034653,0.153362
3,NaN,NaN,M0354,2019-07,12,874,-1400.80,72.833333,-1.602746
4,NaN,NaN,1286A,2018-12,139,9900,17334.75,71.223022,1.750985
...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,S9339,2024-09,71,774,246915.94,10.901408,319.012842
9996,NaN,NaN,S5105,2021-05,20660,225215,8452014.91,10.901016,37.528650
9997,NaN,NaN,S5130,2021-01,90240,983694,55459568.26,10.900864,56.378882
9998,NaN,NaN,J0878,2019-10,30,327,70109.61,10.900000,214.402477


# Look into Home health type CPT codes
- HCPCS = T10**
- https://www.aapc.com/codes/hcpcs-codes-range/409/?srsltid=AfmBOoqCW_2RgDV_1TvqDTG2nLd2tqO9m5DzuXbDOXY4h3rCvKfV_MXI


### output
- one random month's worth of HCPC codes billed
    - verify the ranks are working as intended

In [199]:
# home_health_util = hcpc_by_month_filtered = hcpc_by_month[hcpc_by_month['HCPCS_CODE'].str[:3] == 'T10']
home_health_util = hcpc_by_month[hcpc_by_month['HCPCS_CODE'].str[:3] == 'T10']

# Sort the DataFrame by Category and Sales (descending)
# Sorting first ensures ranks are assigned correctly within groups
home_health_util_sorted = home_health_util.sort_values(['HCPCS_CODE', 'CLAIM_FROM_MONTH'], ascending=[True, False])

# Apply the rank function within each group
# method='min' is similar to SQL's RANK(), which leaves gaps after ties
home_health_util_sorted['PAID_MONTHLY_RANK'] = home_health_util_sorted.groupby('CLAIM_FROM_MONTH')['TOTAL_PAID'].rank(method='min', ascending=False)
home_health_util_sorted['BENEFICIARIES_MONTHLY_RANK'] = home_health_util_sorted.groupby('CLAIM_FROM_MONTH')['TOTAL_UNIQUE_BENEFICIARIES'].rank(method='min', ascending=False)
home_health_util_sorted['CLAIMS_MONTHLY_RANK'] = home_health_util_sorted.groupby('CLAIM_FROM_MONTH')['TOTAL_CLAIMS'].rank(method='min', ascending=False)
display(home_health_util_sorted[home_health_util_sorted.CLAIM_FROM_MONTH=='2021-09'])
# home_health_util_sorted.head()

,HCPCS_CODE,CLAIM_FROM_MONTH,TOTAL_UNIQUE_BENEFICIARIES,TOTAL_CLAIMS,TOTAL_PAID,CLAIMS_PER_BENEFICIARY,PAID_PER_CLAIM,PAID_MONTHLY_RANK,BENEFICIARIES_MONTHLY_RANK,CLAIMS_MONTHLY_RANK
1495,T1000,2021-09,11150,252530,1.036551e+08,22.648430,410.466562,4.0,19.0,8.0
22592,T1001,2021-09,89206,138044,6.214714e+06,1.547474,45.019804,16.0,5.0,13.0
519115,T10019,2021-09,12,14,0.000000e+00,1.166667,0.000000,32.0,32.0,32.0
10753,T1002,2021-09,70406,149808,1.742939e+07,2.127773,116.344844,11.0,7.0,11.0
6827,T1003,2021-09,25625,141263,2.863729e+07,5.512702,202.723204,8.0,13.0,12.0
20248,T1004,2021-09,9363,151073,7.292299e+06,16.135106,48.270034,14.0,20.0,10.0
10154,T1005,2021-09,27745,306849,1.864946e+07,11.059614,60.777325,10.0,12.0,7.0
191008,T1006,2021-09,85,329,6.496138e+04,3.870588,197.451003,29.0,29.0,29.0
74764,T1007,2021-09,7707,8843,6.847426e+05,1.147398,77.433296,23.0,21.0,25.0
289820,T1009,2021-09,13,51,1.330836e+04,3.923077,260.948235,31.0,31.0,30.0


# check to filter on a given month to verify HCPCs percentile logic is working
- to streamline spot checking, filter what you want to see
    - i want to look one 'dimension' of data
    - reduce a variable, reduce the "noise"
    - the more noise you remove, the quicker you can check that the percentile logic is working
    - quicker (and repeatable**) validation
- I want to see how a HCPC claim count(#)/cost($)/beneficiaries(~human lives~) stacks up amongst others in the given month
        - I want to see how
- my data is aggregated by groups by columns, there are aggregations based on one of 3 columns:
    - TOTAL_CLAIMS
    - TOTAL_PAID
    - TOTAL_UNIQUE_BENEFICIARIES

# reset dataframe(s) here

In [200]:
min_clm_mnth = home_health_util_sorted['CLAIM_FROM_MONTH'].min()
# invg_cols = home_health_util_sorted.filter(like='TOTAL_CLAIMS').columns
invg_cols = ['TOTAL_CLAIMS','CLAIMS_MONTHLY_RANK']
print(invg_cols)

# at minimum should have these columns
#   this gives context to what you're looking into
base_cols = home_health_util_sorted[['HCPCS_CODE','CLAIM_FROM_MONTH']].columns


df_filtered_to_month = home_health_util_sorted[home_health_util_sorted.CLAIM_FROM_MONTH==min_clm_mnth]
# #
# final_cols = base_colsc..to_list() + invg_cols..to_list()
#
#
# # # create a list of columns with 'MONTH_PCTILE' as part of the column name
# pctile_cols = home_health_util_sorted.filter(like='MONTH_PCTILE').columns
# remaining_cols = home_health_util_sorted.columns.difference(pctile_cols)

final_cols = base_cols.to_list() + invg_cols#.to_list()

# add layer of context
#   divide by percentile to give depth of records, if they are big chunks -> fewer rows of data
#   one less thing to think about
# since we've filtered down its easier to add a column based on one MAX
#   # no need to account for a group by
#   # efficient code

second_dimension_filtered_df = df_filtered_to_month[final_cols].copy()
invg_max_rank = second_dimension_filtered_df.CLAIMS_MONTHLY_RANK.max() ## hardocded right now, need to add flexibility in case a new level of depth gets added above

second_dimension_filtered_df['CLAIMS_MONTHLY_PCTILE'] = second_dimension_filtered_df['CLAIMS_MONTHLY_RANK'] / invg_max_rank
## include max pctile, why not
#   not as impactful of an addition(i don't like having a column of all the same value, too much "noise" IMO when reviewing data

display(second_dimension_filtered_df)
# T1019 has the most claims out of all other HCPCs codes for this month

['TOTAL_CLAIMS', 'CLAIMS_MONTHLY_RANK']


,HCPCS_CODE,CLAIM_FROM_MONTH,TOTAL_CLAIMS,CLAIMS_MONTHLY_RANK,CLAIMS_MONTHLY_PCTILE
5044,T1000,2018-01,127379,11.0,0.354839
22830,T1001,2018-01,128071,10.0,0.322581
14707,T1002,2018-01,176961,7.0,0.225806
10086,T1003,2018-01,124026,14.0,0.451613
30383,T1004,2018-01,124627,12.0,0.387097
14766,T1005,2018-01,108562,16.0,0.516129
204448,T1006,2018-01,501,29.0,0.935484
145387,T1007,2018-01,3941,26.0,0.838710
233715,T1009,2018-01,430,30.0,0.967742
102921,T1012,2018-01,4425,25.0,0.806452


# output filtered to top percentiles
- write investigative code and production level code `second_dimension_filtered_df
    - investigative : 'hey could you look into X/Y/Z'
    - start there, production-ize to easily iterate across all A-Z
        - that's production code

# TO-DO:
    - average by 2/3 percentiles, see if there's one that is stronger in amount per claimant
    - curiosity

In [203]:
# pull 'base_cols' list from the above code
home_health_util_sorted['PAID_MAX_MONTHLY_RANK'] = (
    home_health_util_sorted.groupby('CLAIM_FROM_MONTH')['PAID_MONTHLY_RANK']
      .transform('max')
)

home_health_util_sorted['BENEFICIARIES_MAX_MONTHLY_RANK'] = (
    home_health_util_sorted.groupby('CLAIM_FROM_MONTH')['BENEFICIARIES_MONTHLY_RANK']
      .transform('max')
)

home_health_util_sorted['CLAIMS_MAX_MONTHLY_RANK'] = (
    home_health_util_sorted.groupby('CLAIM_FROM_MONTH')['CLAIMS_MONTHLY_RANK']
      .transform('max')
)

# changed this from MR to MONTH
home_health_util_sorted['PAID_MONTH_PCTILE'] = home_health_util_sorted['PAID_MONTHLY_RANK'] / home_health_util_sorted['PAID_MAX_MONTHLY_RANK']
home_health_util_sorted['BENEFICIARIES_MONTH_PCTILE'] = home_health_util_sorted['BENEFICIARIES_MONTHLY_RANK'] / home_health_util_sorted['BENEFICIARIES_MAX_MONTHLY_RANK']
home_health_util_sorted['CLAIMS_MONTH_PCTILE'] = home_health_util_sorted['CLAIMS_MONTHLY_RANK'] / home_health_util_sorted['CLAIMS_MAX_MONTHLY_RANK']

home_health_util_sorted['PAID_PER_BENEFICIARY'] = home_health_util_sorted['TOTAL_PAID'] / home_health_util_sorted['TOTAL_UNIQUE_BENEFICIARIES']


display(home_health_util_sorted[home_health_util_sorted['CLAIM_FROM_MONTH']=='2021-09'])

,HCPCS_CODE,CLAIM_FROM_MONTH,TOTAL_UNIQUE_BENEFICIARIES,TOTAL_CLAIMS,TOTAL_PAID,CLAIMS_PER_BENEFICIARY,PAID_PER_CLAIM,PAID_MONTHLY_RANK,BENEFICIARIES_MONTHLY_RANK,CLAIMS_MONTHLY_RANK,PAID_MAX_MONTHLY_RANK,BENEFICIARIES_MAX_MONTHLY_RANK,CLAIMS_MAX_MONTHLY_RANK,PAID_MONTH_PCTILE,BENEFICIARIES_MONTH_PCTILE,CLAIMS_MONTH_PCTILE,PAID_PER_BENEFICIARY
1495,T1000,2021-09,11150,252530,1.036551e+08,22.648430,410.466562,4.0,19.0,8.0,32.0,32.0,32.0,0.12500,0.59375,0.25000,9296.423389
22592,T1001,2021-09,89206,138044,6.214714e+06,1.547474,45.019804,16.0,5.0,13.0,32.0,32.0,32.0,0.50000,0.15625,0.40625,69.666993
519115,T10019,2021-09,12,14,0.000000e+00,1.166667,0.000000,32.0,32.0,32.0,32.0,32.0,32.0,1.00000,1.00000,1.00000,0.000000
10753,T1002,2021-09,70406,149808,1.742939e+07,2.127773,116.344844,11.0,7.0,11.0,32.0,32.0,32.0,0.34375,0.21875,0.34375,247.555442
6827,T1003,2021-09,25625,141263,2.863729e+07,5.512702,202.723204,8.0,13.0,12.0,32.0,32.0,32.0,0.25000,0.40625,0.37500,1117.552700
20248,T1004,2021-09,9363,151073,7.292299e+06,16.135106,48.270034,14.0,20.0,10.0,32.0,32.0,32.0,0.43750,0.62500,0.31250,778.842124
10154,T1005,2021-09,27745,306849,1.864946e+07,11.059614,60.777325,10.0,12.0,7.0,32.0,32.0,32.0,0.31250,0.37500,0.21875,672.173779
191008,T1006,2021-09,85,329,6.496138e+04,3.870588,197.451003,29.0,29.0,29.0,32.0,32.0,32.0,0.90625,0.90625,0.90625,764.251529
74764,T1007,2021-09,7707,8843,6.847426e+05,1.147398,77.433296,23.0,21.0,25.0,32.0,32.0,32.0,0.71875,0.65625,0.78125,88.846846
289820,T1009,2021-09,13,51,1.330836e+04,3.923077,260.948235,31.0,31.0,30.0,32.0,32.0,32.0,0.96875,0.96875,0.93750,1023.720000


In [204]:
# home_health_util_sorted.columns
home_health_util_sorted.CLAIMS_MONTHLY_RANK.unique()

array([ 7.,  8.,  9., 10., 11., 13., 15., 14., 12., 16., 32.,  6.,  5.,
       28., 29., 30., 25., 27., 23., 24., 26., 22., 31., 33., 21., 17.,
       20., 19., 18.,  2.,  4.,  3.,  1., 34.])

In [205]:
# home_health_util_sorted.drop(columns=['MONTH_PCTILE_AVG','MONTH_RANK_AVG'], inplace=True)
# home_health_util_sorted.head()

# FLAG A DIGGING POINT
## oh crap i forgot what i was doing, how do i retrack
## be an interdimensional thinker
## step back

In [206]:
print('qa that column grouping worke')
print(pctile_cols)
print(remaining_cols)

qa that column grouping worke
Index(['PAID_MONTH_PCTILE', 'BENEFICIARIES_MONTH_PCTILE',
       'CLAIMS_MONTH_PCTILE'],
      dtype='object')
Index(['BENEFICIARIES_MAX_MONTHLY_RANK', 'BENEFICIARIES_MONTHLY_RANK',
       'BENEFICIARIES_MR_PCTILE', 'CLAIMS_MAX_MONTHLY_RANK',
       'CLAIMS_MONTHLY_RANK', 'CLAIMS_MR_PCTILE', 'CLAIMS_PER_BENEFICIARY',
       'CLAIM_FROM_MONTH', 'HCPCS_CODE', 'MONTH_RANK_AVG', 'MR_PCTILE_AVG',
       'PAID_MAX_MONTHLY_RANK', 'PAID_MONTHLY_RANK', 'PAID_MR_PCTILE',
       'PAID_PER_CLAIM', 'TOTAL_CLAIMS', 'TOTAL_PAID',
       'TOTAL_UNIQUE_BENEFICIARIES'],
      dtype='object')


check next section
- do the [outputs] look correct?
- As of (2/14/26)  the cell below is
    - averaging the 3 column percentiles
        -   paid / claim count / people
    - filtering
        - percentile < 0.09
        - means the HCPC is in the top 10% of every metric, ranked by most (1) to least (999)
        - if 100 people race 3 times and
                - Jerry comes in 7th, 5th, 3rd
                - Raquel comes in 1st, 75th, 49th
                - which outcome is [preferred]?
                        - to your company?
                        - to your client?
                    - both outcomes can have desirability within them, depending on what you want
                            - maybe you need to find out what caused Raquel to come in 1st

# (TO-DO): to add more context / adaptability
- look at top HCPCs regardless of month
- i just so happened to stumble upon home health
    - i can figure out context enough from google

In [207]:
pctile_cols = home_health_util_sorted.filter(like='MONTH_PCTILE').columns
# month_rank_cols = home_health_util_sorted.filter(like='MONTHLY_RANK').columns

month_rank_cols = ['PAID_MONTHLY_RANK', 'BENEFICIARIES_MONTHLY_RANK', 'CLAIMS_MONTHLY_RANK']
print(pctile_cols)
print(month_rank_cols)

Index(['PAID_MONTH_PCTILE', 'BENEFICIARIES_MONTH_PCTILE',
       'CLAIMS_MONTH_PCTILE'],
      dtype='object')
['PAID_MONTHLY_RANK', 'BENEFICIARIES_MONTHLY_RANK', 'CLAIMS_MONTHLY_RANK']


# todo:
- omit some of the builder columns
- less noise

In [232]:
max_mo = home_health_util_sorted['CLAIM_FROM_MONTH'].max()
min_mo = home_health_util_sorted['CLAIM_FROM_MONTH'].min()
# print(max_mo)
# print(min_mo)

home_health_util_sorted[(home_health_util_sorted.CLAIM_FROM_MONTH == min_mo) | (home_health_util_sorted.CLAIM_FROM_MONTH == max_mo)]

,HCPCS_CODE,CLAIM_FROM_MONTH,TOTAL_UNIQUE_BENEFICIARIES,TOTAL_CLAIMS,TOTAL_PAID,CLAIMS_PER_BENEFICIARY,PAID_PER_CLAIM,PAID_MONTHLY_RANK,BENEFICIARIES_MONTHLY_RANK,CLAIMS_MONTHLY_RANK,PAID_MAX_MONTHLY_RANK,BENEFICIARIES_MAX_MONTHLY_RANK,CLAIMS_MAX_MONTHLY_RANK,PAID_MONTH_PCTILE,BENEFICIARIES_MONTH_PCTILE,CLAIMS_MONTH_PCTILE,PAID_PER_BENEFICIARY,MONTH_PCTILE_AVG,MONTH_RANK_AVG
4322,T1000,2024-12,7805,88589,4.570321e+07,11.350288,515.901673,3.0,12.0,7.0,31.0,31.0,31.0,0.096774,0.387097,0.225806,5855.632712,0.236559,7.333333
5044,T1000,2018-01,5809,127379,3.855342e+07,21.927871,302.666979,6.0,20.0,11.0,31.0,31.0,31.0,0.193548,0.645161,0.354839,6636.842333,0.397849,12.333333
43302,T1001,2024-12,22778,34340,1.949928e+06,1.507595,56.782993,15.0,7.0,11.0,31.0,31.0,31.0,0.483871,0.225806,0.354839,85.605759,0.354839,11.000000
22830,T1001,2018-01,74921,128071,6.116320e+06,1.709414,47.757259,15.0,5.0,10.0,31.0,31.0,31.0,0.483871,0.161290,0.322581,81.636923,0.322581,10.000000
27681,T1002,2024-12,21446,46032,4.412512e+06,2.146414,95.857485,11.0,8.0,10.0,31.0,31.0,31.0,0.354839,0.258065,0.322581,205.749872,0.311828,9.666667
14707,T1002,2018-01,72190,176961,1.146929e+07,2.451323,64.812509,11.0,6.0,7.0,31.0,31.0,31.0,0.354839,0.193548,0.225806,158.876388,0.258065,8.000000
14011,T1003,2024-12,8224,31935,1.228810e+07,3.883147,384.784580,8.0,11.0,13.0,31.0,31.0,31.0,0.258065,0.354839,0.419355,1494.175044,0.344086,10.666667
10086,T1003,2018-01,20853,124026,1.880108e+07,5.947633,151.589845,9.0,13.0,14.0,31.0,31.0,31.0,0.290323,0.419355,0.451613,901.600833,0.387097,12.000000
29262,T1004,2024-12,7035,52332,3.996283e+06,7.438806,76.364044,12.0,13.0,9.0,31.0,31.0,31.0,0.387097,0.419355,0.290323,568.057305,0.365591,11.333333
30383,T1004,2018-01,8129,124627,3.731837e+06,15.331160,29.944049,19.0,16.0,12.0,31.0,31.0,31.0,0.612903,0.516129,0.387097,459.077001,0.505376,15.666667


In [219]:
## create average of percentile values
# combines all percentiles as a fun way to filter data

# # create a list of columns with 'MONTH_PCTILE' as part of the column name
home_health_util_sorted['MONTH_PCTILE_AVG'] = (
    home_health_util_sorted[pctile_cols].mean(axis=1)
)

# also take average rank
home_health_util_sorted['MONTH_RANK_AVG'] = (
    home_health_util_sorted[month_rank_cols].mean(axis=1)
)

filtered_health_utils_df2 = home_health_util_sorted[
    home_health_util_sorted['MONTH_PCTILE_AVG'] < 0.22
]

# display(filtered_health_utils_df)
filtered_health_utils_df2.head()
# filtered_health_utils_df2.head(1000)

,HCPCS_CODE,CLAIM_FROM_MONTH,TOTAL_UNIQUE_BENEFICIARIES,TOTAL_CLAIMS,TOTAL_PAID,CLAIMS_PER_BENEFICIARY,PAID_PER_CLAIM,PAID_MONTHLY_RANK,BENEFICIARIES_MONTHLY_RANK,CLAIMS_MONTHLY_RANK,PAID_MAX_MONTHLY_RANK,BENEFICIARIES_MAX_MONTHLY_RANK,CLAIMS_MAX_MONTHLY_RANK,PAID_MONTH_PCTILE,BENEFICIARIES_MONTH_PCTILE,CLAIMS_MONTH_PCTILE,PAID_PER_BENEFICIARY,MONTH_PCTILE_AVG,MONTH_RANK_AVG
14204,T1005,2024-12,25105,170750,1.204890e+07,6.801434,70.564561,9.0,6.0,5.0,31.0,31.0,31.0,0.290323,0.193548,0.161290,479.940205,0.215054,6.666667
488,T1015,2024-12,1216414,1468053,2.924467e+08,1.206870,199.207195,2.0,1.0,2.0,31.0,31.0,31.0,0.064516,0.032258,0.064516,240.417095,0.053763,1.666667
122,T1015,2024-11,2455604,3165696,6.151738e+08,1.289172,194.324984,2.0,1.0,2.0,32.0,32.0,32.0,0.062500,0.031250,0.062500,250.518335,0.052083,1.666667
81,T1015,2024-10,3118842,4159943,8.044845e+08,1.333810,193.388351,2.0,1.0,2.0,33.0,33.0,33.0,0.060606,0.030303,0.060606,257.943338,0.050505,1.666667
95,T1015,2024-09,2920799,3807733,7.147765e+08,1.303661,187.717078,2.0,1.0,2.0,33.0,33.0,33.0,0.060606,0.030303,0.060606,244.719514,0.050505,1.666667


# (above output) the percentile (average & filter) are being applied correctly
    - percentile average provides more context, is this team a dominant force or are they a one hit wonder?
        - just looking at one percentile is fine too, the intent of this is to dive depeer
## easy investigative jumping off point
    - what changed in bookends of the data'
        - were sales total  $10M the first year and $100M in the latest year? how did that change?
            - what were the highs within that data

# (below cell)
- take min/max rank of monthly claim counts
- output: first / last month's worth of data

In [217]:
filtered_health_utils_df.head()

,HCPCS_CODE,CLAIM_FROM_MONTH,TOTAL_UNIQUE_BENEFICIARIES,TOTAL_CLAIMS,TOTAL_PAID,CLAIMS_PER_BENEFICIARY,PAID_PER_CLAIM,PAID_MONTHLY_RANK,BENEFICIARIES_MONTHLY_RANK,CLAIMS_MONTHLY_RANK,PAID_MAX_MONTHLY_RANK,BENEFICIARIES_MAX_MONTHLY_RANK,CLAIMS_MAX_MONTHLY_RANK,PAID_MONTH_PCTILE,BENEFICIARIES_MONTH_PCTILE,CLAIMS_MONTH_PCTILE,PAID_PER_BENEFICIARY,MONTH_PCTILE_AVG,MONTH_RANK_AVG
488,T1015,2024-12,1216414,1468053,2.924467e+08,1.206870,199.207195,2.0,1.0,2.0,31.0,31.0,31.0,0.064516,0.032258,0.064516,240.417095,0.053763,1.666667
122,T1015,2024-11,2455604,3165696,6.151738e+08,1.289172,194.324984,2.0,1.0,2.0,32.0,32.0,32.0,0.062500,0.031250,0.062500,250.518335,0.052083,1.666667
81,T1015,2024-10,3118842,4159943,8.044845e+08,1.333810,193.388351,2.0,1.0,2.0,33.0,33.0,33.0,0.060606,0.030303,0.060606,257.943338,0.050505,1.666667
95,T1015,2024-09,2920799,3807733,7.147765e+08,1.303661,187.717078,2.0,1.0,2.0,33.0,33.0,33.0,0.060606,0.030303,0.060606,244.719514,0.050505,1.666667
90,T1015,2024-08,3089005,4026213,7.586311e+08,1.303401,188.422995,2.0,1.0,2.0,33.0,33.0,33.0,0.060606,0.030303,0.060606,245.590769,0.050505,1.666667


In [222]:
# filtered_health_utils_df2.CLAIMS_MONTHLY_RANK.unique()
filtered_health_utils_df.CLAIMS_MONTHLY_RANK.unique() ## bad table

array([2., 1.])

In [195]:
filtered_health_utils_df.tail()

,HCPCS_CODE,CLAIM_FROM_MONTH,TOTAL_UNIQUE_BENEFICIARIES,TOTAL_CLAIMS,TOTAL_PAID,CLAIMS_PER_BENEFICIARY,PAID_PER_CLAIM,PAID_MONTHLY_RANK,BENEFICIARIES_MONTHLY_RANK,CLAIMS_MONTHLY_RANK,PAID_MAX_MONTHLY_RANK,BENEFICIARIES_MAX_MONTHLY_RANK,CLAIMS_MAX_MONTHLY_RANK,PAID_MONTH_PCTILE,BENEFICIARIES_MONTH_PCTILE,CLAIMS_MONTH_PCTILE,MONTH_PCTILE_AVG,MONTH_RANK_AVG
76,T1019,2018-05,501746,9924407,8.197985e+08,19.779743,82.604283,1.0,2.0,1.0,30.0,30.0,30.0,0.033333,0.066667,0.033333,0.044444,1.333333
83,T1019,2018-04,500569,9584214,7.816811e+08,19.146639,81.559228,1.0,2.0,1.0,30.0,30.0,30.0,0.033333,0.066667,0.033333,0.044444,1.333333
92,T1019,2018-03,474934,9015637,7.337889e+08,18.982926,81.390686,1.0,2.0,1.0,30.0,30.0,30.0,0.033333,0.066667,0.033333,0.044444,1.333333
113,T1019,2018-02,469751,8454437,6.618927e+08,17.997699,78.289389,1.0,2.0,1.0,30.0,30.0,30.0,0.033333,0.066667,0.033333,0.044444,1.333333
96,T1019,2018-01,472744,9047566,7.145200e+08,19.138405,78.973726,1.0,2.0,1.0,31.0,31.0,31.0,0.032258,0.064516,0.032258,0.043011,1.333333


In [223]:
remaining_cols = home_health_util_sorted.columns.difference(pctile_cols)
total_cols = remaining_cols.tolist() + pctile_cols.tolist()

max_clm_mo = filtered_health_utils_df2.CLAIMS_MONTHLY_RANK.max()
min_clm_mo = filtered_health_utils_df2.CLAIMS_MONTHLY_RANK.min()

print(total_cols)
print(max_clm_mo)
print(min_clm_mo)

['BENEFICIARIES_MAX_MONTHLY_RANK', 'BENEFICIARIES_MONTHLY_RANK', 'CLAIMS_MAX_MONTHLY_RANK', 'CLAIMS_MONTHLY_RANK', 'CLAIMS_PER_BENEFICIARY', 'CLAIM_FROM_MONTH', 'HCPCS_CODE', 'MONTH_PCTILE_AVG', 'MONTH_RANK_AVG', 'PAID_MAX_MONTHLY_RANK', 'PAID_MONTHLY_RANK', 'PAID_PER_BENEFICIARY', 'PAID_PER_CLAIM', 'TOTAL_CLAIMS', 'TOTAL_PAID', 'TOTAL_UNIQUE_BENEFICIARIES', 'PAID_MONTH_PCTILE', 'BENEFICIARIES_MONTH_PCTILE', 'CLAIMS_MONTH_PCTILE']
8.0
1.0


# should return the first / last elements of the dataframe, but isnt

In [224]:

# ensures pcitle columns come at the end of the DF (cleanliness, automated)
# dimensional_colums_df = filtered_health_utils_df[total_cols]

# dimensional_colums_df[(dimensional_colums_df['CLAIM_FROM_MONTH'] == max_clm_mo) | (dimensional_colums_df['CLAIM_FROM_MONTH'] == min_clm_mo)]
# bookend_mos_df = filtered_health_utils_df2[(filtered_health_utils_df2['CLAIM_FROM_MONTH'] == max_clm_mo) | (filtered_health_utils_df2['CLAIM_FROM_MONTH'] == min_clm_mo)].copy()
# display(dimensional_colums_df)
# dimensional_colums_df.columns

# bookend_mos_df.columns
display(bookend_mos_df)

,HCPCS_CODE,CLAIM_FROM_MONTH,TOTAL_UNIQUE_BENEFICIARIES,TOTAL_CLAIMS,TOTAL_PAID,CLAIMS_PER_BENEFICIARY,PAID_PER_CLAIM,PAID_MONTHLY_RANK,BENEFICIARIES_MONTHLY_RANK,CLAIMS_MONTHLY_RANK,PAID_MAX_MONTHLY_RANK,BENEFICIARIES_MAX_MONTHLY_RANK,CLAIMS_MAX_MONTHLY_RANK,PAID_MONTH_PCTILE,BENEFICIARIES_MONTH_PCTILE,CLAIMS_MONTH_PCTILE,PAID_PER_BENEFICIARY,MONTH_PCTILE_AVG,MONTH_RANK_AVG


# total paid by HCPCs code, sorted descending
    - the most [money spent] on any [code]
    - big numbers up top, descend to small numbers on bottom

In [32]:
hcpc_by_month.sort_values(by=['TOTAL_PAID'], ascending=False,inplace=True)
display(hcpc_by_month)
# hcpc_by_month[hcpc_by_month['HCPCS_CODE']=='T1019']

,HCPCS_CODE,CLAIM_FROM_MONTH,TOTAL_UNIQUE_BENEFICIARIES,TOTAL_CLAIMS,TOTAL_PAID,CLAIMS_PER_BENEFICIARY,PAID_PER_CLAIM
0,T1019,2024-05,820816,16668403,2.223192e+09,20.307113,133.377654
1,T1019,2024-07,775502,15774135,2.209226e+09,20.340547,140.053683
2,T1019,2024-03,815880,16268673,2.158420e+09,19.940032,132.673366
3,T1019,2024-06,773575,15206151,2.118378e+09,19.656983,139.310586
4,T1019,2024-04,815162,16059735,2.099838e+09,19.701280,130.751701
...,...,...,...,...,...,...,...
535917,M0360,2019-06,73,2926,-6.630010e+03,40.082192,-2.265895
535918,M0360,2019-08,73,3132,-6.812320e+03,42.904110,-2.175070
535919,M0360,2019-05,74,3068,-6.844470e+03,41.459459,-2.230922
535920,M0360,2019-01,73,3132,-6.973120e+03,42.904110,-2.226411


# Paid per claim, by code, descending
    - not necessarily most dollars
        - what costs us $100k every time they bill us vs $10 every time they bill us
    - adds a potential investigative jumping off point

In [33]:
hcpc_by_month.sort_values(by=['PAID_PER_CLAIM'], ascending=False,inplace=True)
display(hcpc_by_month)

,HCPCS_CODE,CLAIM_FROM_MONTH,TOTAL_UNIQUE_BENEFICIARIES,TOTAL_CLAIMS,TOTAL_PAID,CLAIMS_PER_BENEFICIARY,PAID_PER_CLAIM
44525,J2326,2021-09,13,14,1857803.10,1.076923,132700.221429
41685,J2326,2023-12,12,16,2088864.05,1.333333,130554.003125
42579,J2326,2023-03,15,16,2010586.73,1.066667,125661.670625
28827,J2326,2020-06,26,33,4102351.20,1.269231,124313.672727
29716,J2326,2020-04,29,32,3886260.90,1.103448,121445.653125
...,...,...,...,...,...,...,...
535845,11047,2020-07,21,44,-482.25,2.095238,-10.960227
535867,M0299,2020-01,23,90,-1355.98,3.913043,-15.066444
535904,M0118,2020-03,14,137,-4598.65,9.785714,-33.566788
535850,M0145,2020-09,21,21,-845.90,1.000000,-40.280952


# Claims per Beneficiary, by code, descending
    - are 5 people the cause of 5,000 claims?
    - are members/patients regularly being billed for this within a month?
        - doesn't paint the full picture
            - if something is billed 10k times but we only pay $1 per instance
            - is that as significant as
                - $50k/instance billed 2,000 times?
                    - probably

In [34]:
hcpc_by_month.sort_values(by=['CLAIMS_PER_BENEFICIARY'], ascending=False,inplace=True)
display(hcpc_by_month)

,HCPCS_CODE,CLAIM_FROM_MONTH,TOTAL_UNIQUE_BENEFICIARIES,TOTAL_CLAIMS,TOTAL_PAID,CLAIMS_PER_BENEFICIARY,PAID_PER_CLAIM
215534,1286A,2022-02,355,309532,43321.27,871.921127,0.139957
122930,1286Z,2022-02,1906,1607071,229965.29,843.164218,0.143096
251262,1286C,2022-02,202,157971,24226.68,782.034653,0.153362
535871,M0354,2019-07,12,874,-1400.80,72.833333,-1.602746
272850,1286A,2018-12,139,9900,17334.75,71.223022,1.750985
...,...,...,...,...,...,...,...
489109,Q2038,2022-05,70,70,0.02,1.000000,0.000286
457136,57800,2024-07,13,13,337.75,1.000000,25.980769
388730,L0635,2018-12,14,14,2357.00,1.000000,168.357143
453892,K0019,2024-12,15,15,389.71,1.000000,25.980667


,Code,Description,Category
0,86890,Autologous blood process,CLINICAL LABORATORY SERVICES
1,86891,Autologous blood op salvage,CLINICAL LABORATORY SERVICES
2,86927,Plasma fresh frozen,CLINICAL LABORATORY SERVICES
3,86930,Frozen blood prep,CLINICAL LABORATORY SERVICES
4,86931,Frozen blood thaw,CLINICAL LABORATORY SERVICES
